# **CDS Project: Part 1**

*Institute of Software Security (E22)*  
*Hamburg University of Technology*  
*SoSe 2023*

## Learning objectives
---

- Use a basic Machine Learning (ML) pipeline with pre-trained models.
- Build your own data loader.
- Load and run a pre-trained ML model.
- Evaluate the performance of an ML model.
- Calculate and interpret performance metrics.

## Materials
---

- Lecture Slides 1, 2, and 3.
- PyTorch Documentation: [Datasets and Data Loaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) 


## Project Description
---

In this project, you are given an ML model that is pre-trained on a vulnerability dataset. The dataset consists of code samples labeled with True or False flags, depending on the presence and absense of a vulnerability. Your goal is to use the pre-trained model to predict if the code samples in the validation set contain vulnerabilities or not and analyse the results. Please proceed to the below tasks. 

###*Task 1*

Build a data loader for the validation dataset present in the following path: "*data_students/student_dataset.hdf5*". You will be using this dataset to validate the performance of the ML model. The dataset is in HDF5 binary data format. This format is used to store large amount of data. Make sure that you import and familiarise yourself with the right Python libraries to handle HDF5 files. 


In [8]:
# TODO: import the necessary libraries to load the data from the specified path.
import h5py
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


# Load the dataset
with h5py.File("student_dataset.hdf5", "r") as f:
   print("Keys in dataset:", list(f.keys()))
   sources = f["source"][:]
   labels = f["labels"][:]


Using cpu device
Keys in dataset: ['labels', 'source', 'vectors']


###*Task 2*

Generate a table with 10 random samples from the dataset and show their corresponding labels.

In [9]:
# TODO: display 10 random samples from the loaded dataset
import pandas as pd
import numpy as np

# Decode byte strings to regular strings
decoded_sources = [s.decode("utf-8") for s in sources]

# Create DataFrame
df = pd.DataFrame({"source": decoded_sources, "label": labels})

# Display 10 random samples
sample_df = df.sample(n=10, random_state=42).reset_index(drop=True)
print(sample_df)


                                              source  label
0  main(int argc, char **argv)\n{\n  char        ...  False
1  clear() {\n  for (FL_BLINE* l = first; l;) {\n...  False
2  GAsetwidgetattr (Gwidget_t *widget, int attrn,...  False
3  gamgi_mesa_text_font_start (void)\n{\nint i;\n...   True
4  hwloc_topology_set_distance_matrix(hwloc_topol...   True
5  ivl_stmt_rval(ivl_statement_t net)\n{\n      s...  False
6  prepareDone(Batch& batch, int status)\n{\n    ...  False
7  xsltParseStylesheetDoc(xmlDocPtr doc) {\n    x...  False
8  is_rtcp_time (RTPSession * sess, GstClockTime ...   True
9  ext2fs_close2(ext2_filsys fs, int flags)\n{\n\...  False



Inspect the dataset and answer the following questions:
1.  How many samples are in the dataset?
2. How many positive examples (vulnerability-labeled instances) are in the dataset?
3. What is the vulnerable/non-vulnerable ratio?

In [23]:
# TODO: inspect and understand the loaded dataset

# Total number of samples
ts = len(labels)

# Positive examples (assuming True or 1 means vulnerable)
pos = np.sum(labels == True)

# Negative examples
neg = ts - pos

# Vulnerable / Non-vulnerable ratio
ratio = pos / neg if neg > 0 else float('inf')

# Display the results
print(f"Total samples: {ts}")
print(f"Vulnerable (positive) samples: {pos}")
print(f"Vulnerable / Non-vulnerable ratio: {ratio:.2f}")

Total samples: 1000
Vulnerable (positive) samples: 283
Vulnerable / Non-vulnerable ratio: 0.39


###*Task 4*

Load and run the following pre-trained neural network model called VulnPredictionModel. 

``` python 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

```

In [11]:
import torch
from torch import nn

# Define the model class
class VulnPredictModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(768, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        pred = self.linear_stack(x)
        return pred

# Set device (assumes CUDA if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model
model = VulnPredictModel().to(device)

# Load pretrained weights
model.load_state_dict(torch.load("C:/Users/amrit/Downloads/Cybersecurity/Lab 1/lab/model_2023-03-28_20-03.pth", map_location=device))
model.eval()

print("Model loaded and ready for inference!")
print(model)

✅ Model loaded and ready for inference!
VulnPredictModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=768, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


###*Task 5*

Make a prediction on the provided dataset and compute the following values:
- True Positives
- True Negatives
- False Positives
- False Negatives

In [18]:
import h5py

# Open the HDF5 file and list keys
with h5py.File('student_dataset.hdf5', 'r') as f:
    print("Keys inside the file:", list(f.keys()))


Keys inside the file: ['labels', 'source', 'vectors']


In [19]:
# TODO: make the prediction for all the samples in the validation set.
import torch
from torch.utils.data import Dataset
import h5py

class HDF5ValidationDataset(Dataset):
    def __init__(self, file_path):
        super().__init__()
        self.file_path = file_path
        
        # Open the file
        self.h5file = h5py.File(file_path, 'r')
        
        
        self.inputs = self.h5file['vectors'][:]    
        self.labels = self.h5file['labels'][:]  

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_tensor = torch.tensor(self.inputs[idx], dtype=torch.float32)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.float32)
        return input_tensor, label_tensor

device = "cuda" if torch.cuda.is_available() else "cpu"

model = VulnPredictModel().to(device)

val_dataset = HDF5ValidationDataset("student_dataset.hdf5")
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

model.eval()
tp = fp = tn = fn = 0

with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        preds = (model(x).squeeze() > 0.5).float()

        tp += ((preds == 1) & (y == 1)).sum().item()
        tn += ((preds == 0) & (y == 0)).sum().item()
        fp += ((preds == 1) & (y == 0)).sum().item()
        fn += ((preds == 0) & (y == 1)).sum().item()

print(f"True Positive: {tp}, True Negative: {tn}, False Positive: {fp}, False Negative: {fn}")


# todo: compute true positives, true negatives, false postives and false negatives.

True Positive: 13, True Negative: 670, False Positive: 47, False Negative: 270


### *Task 6*

Compute the corresponding performance metrics **manually** (do not use PyTorch's predefined metrics):
- Accuracy
- Precision
- Recall
- F1

In [20]:
# TODO: calculate accuracy

# TODO: calculate precision

# TODO: calculate recall

# TODO: calculate F1-score
# Compute manually
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) != 0 else 0
recall = tp / (tp + fn) if (tp + fn) != 0 else 0
f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

# Print nicely
print(f"\nPerformance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")



Performance Metrics:
Accuracy: 0.6830
Precision: 0.2167
Recall: 0.0459
F1 Score: 0.0758


### *Task 7*

Based on your performance metrics, answer the following questions:

- Explain the impact of accuracy vs. F1 score.
- In this particular problem, which metric one should focus more on?
- Is there a better metric suitable for the use case of vulnerability prediction? Why?
